In [ ]:
!pip install gdal-utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 338.1/338.1 kB 4.4 MB/s eta 0:00:00


In [ ]:
!pip install pystac

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.9/182.9 kB 3.8 MB/s eta 0:00:00


In [ ]:
from osgeo import gdal

In [ ]:
def split_tiff(input_tiff, output_folder, chip_size):
  dataset= gdal.Open(input_tiff)
  width, height= dataset.RasterXSize, dataset.RasterYSize

  for i in range(0, width, chip_size):
    for j in range(0, height, chip_size):
      w= min(chip_size, width-i)
      h= min(chip_size, height-j)
      gdal.Translate(
          f"{output_folder}/chip_{i}_{j}.tif",
          dataset,
          srcWin= [i,j, w, h],
          format="GTiff"
      )

#using
split_tiff("Orthomosaic.rgb1108_75.tif", "sample data", 1024)

In [ ]:
import os
import pystac
from datetime import datetime

In [ ]:
#Function for creating a STAC item
def create_stac_item(chip_path, bbox):
  item= pystac.Item(
      id= os.path.basename(chip_path),
      geometry= {
          "type": "Polygon",
          "coordinates": [[
              [bbox[0], bbox[3]],
              [bbox[2], bbox[3]],
              [bbox[2], bbox[1]],
              [bbox[0], bbox[1]],
              [bbox[0], bbox[3]]
          ]]
      },
      bbox= bbox,
      datetime= datetime.utcnow(),
      properties={}
  )

  asset= pystac.Asset(href=chip_path, media_type=pystac.MediaType.GEOTIFF)
  item.add_asset('data', asset)
  return item

#Function for getting bounding box from a TIFF file
def get_bbox(tiff_path):
  ds= gdal.Open(tiff_path)
  gt= ds.GetGeoTransform()
  minx= gt[0]
  maxy= gt[3]
  maxx= minx + (gt[1] * ds.RasterXSize)
  miny= maxy + (gt[5] * ds.RasterYSize)
  return [minx, miny, maxx, maxy]

#Reading TIFF files in the 'sample data' folder
chip_paths= [os.path.join("sample data", f) for f in os.listdir("sample data") if f.endswith('.tif')]


#Creating STAC items
items= []
for chip_path in chip_paths:
  bbox= get_bbox(chip_path)
  item = create_stac_item(chip_path, bbox)
  items.append(item)

#Creating a STAC collection
collection= pystac.Collection(
    id="sample_data_collection",
    description="A collection of orthomosaic chips",
    extent=pystac.Extent(
        pystac.SpatialExtent([item.bbox for item in items]),
        pystac.TemporalExtent([[item.datetime, item.datetime] for item in items])
    ),
    license="CC-BY-4.0"
)

for item in items:
  collection.add_item(item)

#Saving the collection to the folder
output_dir= "STAC folder"
os.makedirs(output_dir, exist_ok=True)
collection.normalize_hrefs(output_dir)
collection.save(catalog_type=pystac.CatalogType.SELF_CONTAINED)

print("STAC collection created and saved.")

STAC collection created and saved.
